# Латентно-семантический анализ (Latent Semantic Analysis или LSA)

В этом тюториале мы:
- познакомимся со стандартным датасетом _20 Newsgroups_
- научимся использовать метод латентно-семантическго анализа (LSA) для представления текстов в виде плотных (dense) эмбеддингов, компонентами которых являются скрытые признаки-тематики
- научимся использовать LSA-эмбеддинги для решения задач семантической близости и текстового ранжирования

При подготовке этого тюториала использовались материалы из (отличной) книги _Natural Language Processing in Action_ (авторы _Hobson Lane_ и _Maria Dyshel_): https://www.goodreads.com/book/show/59694556-natural-language-processing-in-action-second-edition

Импортируем модули которые нам понадобятся впоследствии:

In [ ]:
from timeit import default_timer as timer

import numpy as np
import pandas as pd

from sklearn import datasets
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise

## Датасет 20 Newgroups

Наш тюториал будет основан на классическом датасете _20 Newsgroups_.

Этот датасет был собран еще в начале 90-х годов, и представляет из себя коллекцию из примерно 20000 текстовых сообщений, отправленных в 20 разных каналах (newsgroups) в сети Usenet (https://en.wikipedia.org/wiki/Usenet_newsgroup). Название канала (напр. _sci.space_, т.е. канал про космос) используется как метка класса.

Другими словами, это небольшой текстовый датасет для мультиклассовой классификации, всего 20 классов (по числу каналов).

Этот датасет приобрел очень большую популярность в мире NLP, и широко используется для иллюстрации работы различных алгоритмов, моделей и т.п.

Кроме того, этим датасетом очень удобно пользоваться, т.к. он доступен "из коробки" в библиотеке _scikit-learn_: https://scikit-learn.org/stable/datasets/real_world.html#the-20-newsgroups-text-dataset

Попробуем загрузить датасет:

In [ ]:
# Filter e-mail headers, signature blocks and quotes
data_remove = ('headers', 'footers', 'quotes')

# Fetcher options
data_kwargs = { 'remove': data_remove, 'shuffle': True, 'random_state': 22 }

# Fetch train subset. 
# Bunch contains:
#   - data          -- list of text documents
#   - filenames     -- list of sample filenames
#   - target        -- numeric target in [0, 19]
#   - target_names  -- list of target names
train_bunch = datasets.fetch_20newsgroups(subset="train", **data_kwargs)
print(f"fetched 20 newsgroups dataset: num_docs = {len(train_bunch.data)}")

Обратите внимание, что:
- датасет поделен на train и test части, нас дальше будет интересовать только train
- мы скачиваем очищенный вариант датасета с удаленной из сообщений служебной информацией, такой как заголовки писем и т.п.

Всего у нас скачалось ~11 тыс. документов.

Посмотрим теперь на метки классов:

In [ ]:
print(train_bunch.target)
print(train_bunch.target_names)

Видно, что таргеты доступны как в виде текстов (названий каналов), так и в виде числовых идентификаторов.

Посмотрим теперь на типичные тексты сообщений:

In [ ]:
docs = train_bunch.data
print(docs[0])

К какому классу относится наше сообщение?

Это легко понять следующим образом:

In [ ]:
y_0 = train_bunch.target[0]
print(train_bunch.target_names[y_0])

Как (наверное) нетрудно было догадаться, это сообщение про хоккей, отправленное в канал _rec.sport.hockey_

## LSA-эмбеддинги

Попробуем теперь воспользоваться методом LSA и получить эмбеддинги наших текстов.

Первым делом надо превратить текст в матрицу термин-документ, т.е. получить для него разреженное признаковое представление.<br>
Сразу обратим внимание, что в качестве весов в ячейках матрицы термин-документ можно бы было использовать любой из методов взвешивания, изученных нами в лекции про модели векторого пространства, например:
- бинарные веса 1 (слово есть в документе) или 0 (слова нет в документе), которые проще всего получить с помощью класса _CountVectorizer_ из библиотеки _scikit-learn_
- TFы, т.е. частоты слова в документах
- TF-IDFы с разными способами нормализации, сглаживания и т.п.

На практике, как правило, в качестве признаков лучше всего работают TF-IDFы, поэтому мы дальше будем использовать именно их.<br>
Векторизуем наши тексты с помощью класса _TfidfVectorizer_:

In [ ]:
# Prepare vectorizer
vectorizer = TfidfVectorizer(min_df=0.002, max_df=0.5, stop_words='english', decode_error='ignore')

# Vectorize
start = timer()
X_docs = vectorizer.fit_transform(docs)
print(type(X_docs))
print(f"vectorized: X_docs.shape = {X_docs.shape} elapsed = {timer() - start:.3f}")

При векторизации мы сразу выкидываем стоп-слова (параметр _stop_words_), а также слишком редкие (параметр _min_df_) и слишком часто встречающиеся слова (_max_df_).

В результате у нас получилась матрица термин-документ размером 11314x5358, т.е. 11314 документов и 5358 терминов-слов.

Обратим внимание, что при желании мы могли бы использовать тут стемминг, лемматизацию или какую-то кастомную токенизацию, но не делаем этого чтобы не загромождать тюториал.

Наша матрица содержит TF-IDF весах, причем все векторы документы нормализованы с использованием L2-нормы (это дефолтное поведение векторизатора):

In [ ]:
print(X_docs.toarray())

Видно, что матрица крайне разреженная, и действительно:

In [ ]:
print(X_docs.nnz)

Т.е. у нас заполнено только 516701 / (11314 * 5358) = 0.0085 элементов!

Посмотрим сразу и на термины, которые использует векторизатор:

In [ ]:
vocab = vectorizer.get_feature_names_out()
docs_df = pd.DataFrame(X_docs.toarray(), columns=vocab)
docs_df.head()

Теперь начинается самое интересное.

Применим к нашей матрице SVD-разложение:

In [ ]:
# Prepare SVD
svd = TruncatedSVD(n_components=20, n_iter=100, random_state=22)

# Run SVD
start = timer()
E_docs = svd.fit_transform(X_docs)
print(E_docs.round(3))
print(f"decomposed: E_docs.shape = {E_docs.shape} elapsed = {timer() - start:.3f}")

Мы используем класс _TruncatedSVD_ из библиотеки _scikit-learn_, а в качестве параметра _n_components_ передаем ему желаемый размер нашего скрытого пространства.

Размер скрытого пространства является, по сути, гиперпараметром нашего метода и его можно попробовать подобрать под целевую задачу, в которой мы впоследствии будем использовать наши эмбеддинги. На практике, как правило, хорошо работают эмбеддинги размером 100-300 элементов.

В данном примере мы будем использовать _n_components_ равное 20, потому что:
- нам заранее известно, что у нас всего 20 классов, т.е. кажется что 20 -- это минимальное количество тематик, которыми можно описать нашу коллекцию
- с другой стороны, хотим показать, что даже такое радикальное уменьшение размерности (с 5358 до 20!) позволит не только использовать получившиеся эмбеддинги в задаче ранжирования, но и наделит их "суперспособностю" к определению степени семантической близости, которой не обладали исходные sparse-эмбеддинги. 

Посмотрим на получившиеся в результате SVD-разложения сингулярные значения:

In [ ]:
print(svd.singular_values_)

Мы видим, что они убывают, как и должно быть, причем самый главный компонент выглядит особенно "мощным".

Теперь попробуем получить матрицу термин-тематика (а не термин-документ как раньше). Для этого мы воспользуемся матрицей _Vt_ нашего разложения _X = USVt_.

Обратите внимание, что _TfidfVectorizer_ выдал нам _транспонированную_ матрицу термин-документ, поэтому, в отличие от того варианта разложения которое мы разбирали на лекции, тут у нас "все наоборот", т.е. матрица _U_ соответствует документам, а матрица _Vt_ терминам.

In [ ]:
Vt = svd.components_.T
term2topic = pd.DataFrame(data=Vt, index=vocab, columns = [f'topic_{r}' for r in range(0, Vt.shape[1])])
term2topic.tail()

Мы видим, насколько каждый термин соответсвует каждой из 20 тематик.

Возьмем теперь 0-ю (самую сильную, т.е. с самым большим сингулярным значением) тематику и посмотрим, какие термины наиболее тестно с ней связаны, т.е. по сути получим облако слов, лучше всего описывающих данную тематику:

In [ ]:
concept_series = term2topic[f'topic_0']
concept_series = concept_series.sort_values(ascending=False)
concept_series[:10]

Признаемся честно, тут ничего не понятно :-)

Однако не будем расстраиваться раньше времени и посмотрим на следующую по силе тематику:

In [ ]:
concept_series = term2topic[f'topic_1']
concept_series = concept_series.sort_values(ascending=False)
concept_series[:10]

Видим, что у нас отчетливо выделилась тематика, связанная с религией, что неудивительно с учетом того, что в датасете были использованы сообщения из каналов _alt.atheism_, _soc.religion.christian_ и _talk.religion.misc_!

Посмотрим еще на какую-нибудь тематику:

In [ ]:
concept_series = term2topic[f'topic_5']
concept_series = concept_series.sort_values(ascending=False)
concept_series[:10]

Тут у нас, очевидно, что-то околокомпьютерное.

Таким образом, мы видим, что метод LSA действительно позволяет выделять осмысленные тематики, несмотря на то, что мы использовали эмбеддинги размером всего 20 компонент!

## Ранжирование с помощью LSA

Теперь перейдем к самому интересному: попробуем ранжировать наши документы, используя в качестве ранков косинусное расстояние между LSA-векторами запросов и документов, и сравним с тем что получилось бы с использованием обычных TF-IDF-векторов.

Допустим, у нас есть текстовый запрос query.

Напишем две фукнции _search_sparse(query)_ и _search_dense(query)_, которые находят топ-К ближайших к запросу документов с использованием, соответственно, TF-IDF и LSA векторов.

Начнем с _search_sparse(query)_, она:
- векторизует наш запрос с помощью обученного ранее векторизатора
- считает попарную близость между TF-IDF-вектором запроса и TF-IDF-векторами документов
- выводит на экран запрос и ранжированный список документов

In [ ]:
def search_sparse(query):
        # Vectorize query
        X_query = vectorizer.transform([query])
        print(f"vectorized query: X_query.shape = {X_query.shape}")

        # Query-docs similarity
        S = pairwise.cosine_similarity(X_query, X_docs)
        print(f"got similarities: S.shape = {S.shape}")

        # Rank docs
        scores = S[0]
        indexes = np.argsort(scores)[::-1]
        ranked_docs = np.array(docs)[indexes]
        ranked_doc_scores = scores[indexes]

        # Output query and list of ranked docs
        print(f"query = '{query}'")
        for i, doc in enumerate(ranked_docs[0:3]):
            score = ranked_doc_scores[i]
            print(f"SPARSE: [{i}]: doc = '{doc}' score = {score:.3f}")

И, аналогично, напишем функцию _search_dense(query)_, которая делает все то же самое, но уже в пространстве LSA-эмбеддингов:

In [ ]:
def search_dense(query):
        # Vectorize query
        X_query = vectorizer.transform([query])
        print(f"vectorized query: X_query.shape = {X_query.shape}")

        # Embed query
        E_query = svd.transform(X_query)
        print(f"SVD-vectorized query: E_query.shape = {E_query.shape}")

        # Query-docs similarity
        S = pairwise.cosine_similarity(E_query, E_docs)
        print(f"got latent similarities: S.shape = {S.shape}")

        # Rank docs
        scores = S[0]
        indexes = np.argsort(scores)[::-1]
        ranked_docs = np.array(docs)[indexes]
        ranked_doc_scores = scores[indexes]

        # Output query and list of ranked docs
        print(f"query = '{query}'")
        for i, doc in enumerate(ranked_docs[0:3]):
            score = ranked_doc_scores[i]
            print(f"DENSE: [{i}]: doc = '{doc}' score = {score:.3f}")

Применим наши функции к запросу "mars", начем со sparse-варианта:

In [ ]:
search_sparse("mars")

Видим, что в топе выдачи вполне себе релевантные документы про планету Марс, как и ожидалось.

Но что же выдаст dense-вариант?

In [ ]:
search_dense("mars")

Мы видим, что находятся документы про космические исследования, причем ни в одном из документов в топ-3 нет слова "mars"!

Очевидно, что это не может быть случайностью, а это значит что мы действительно смогли сматчить запросы и документы не по факту наличия ключевых слов, а по их семантической близости!

Попробуем еще один запрос:

In [ ]:
search_sparse("penguins")

В топе документы про хоккей, т.к. _Pittsburgh Penguins_ -- это одна из комманд NHL.

Повторим поиск с использованием LSA-эмбеддингов:

In [ ]:
search_dense("penguins")

На 1-м месте у нас тоже документ про хоккей (т.к. _Toronto Maple Leafs_ и _Detroit Red Wings_ -- это тоже команды NHL), на 2-м просто что-то похожее на хоккей, а вот 3-й документ уже кажется про бейсбол (судя по слову _bat_ т.е. "бита"), но это тоже спорт, т.е. и тут мы смогли распознать какие-то оттенки семантики.